In [1]:
#https://bitcointalk.org/index.php?topic=4992632.msg45031167#msg45031167
#https://github.com/sipa/bech32/blob/master/ref/python/segwit_addr.py

In [2]:
!pip install ecdsa bech32

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 142 kB 5.3 MB/s 


In [3]:
import hashlib, ecdsa, bech32

In [4]:
# Step0: ECDSA Private Key")
G = ecdsa.curves.SECP256k1.generator
k = 1 #secret : 1<=k<0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141 *)
print("0 ECDSA Private Key: ", k)
print("------------------------------------------------------")
# Step1: Generate ECDSA Public Key
P = k * G
x_P = P.x()
y_P = P.y()
public_key = '04' + format(x_P, 'x')+ format(y_P, 'x')
parity_y = ('02' if y_P % 2 == 0 else '03')
compressed_public_key = parity_y + format(x_P, 'x')
step1 = compressed_public_key
print("1 ECDSA Public Key: ", public_key)
print(" compressed public key", compressed_public_key)
print("------------------------------------------------------")
# Step2: SHA256 of public key
hash256FromECDSAPublicKey = hashlib.sha256(bytes.fromhex(step1)).hexdigest()
step2 = hash256FromECDSAPublicKey
print("2 SHA256(ECDSA Public Key): ", step2)
print("------------------------------------------------------")
# Step3: RIDEMP160 of step 2
ridemp160FromHash256 = hashlib.new('ripemd160', bytes.fromhex(step2)).hexdigest()
step3 = ridemp160FromHash256
print("3 RIDEMP160(SHA256(ECDSA Public Key)): ", step3) 
print(" RIDEMP160 as array of 8-bit unsigned integers:", [b for b in bytes.fromhex(step3)])
print("------------------------------------------------------")
# Step4: Bech32 bit conversion: array of 8-bit unsigned integers  -> array of 5-bit unsigned integers
fivebit_witprog = bech32.convertbits([b for b in bytes.fromhex(step3)], 8, 5)
step4 = fivebit_witprog
print("4 RIDEMP160 as array of 5-bit unsigned integers:", step4)
print("------------------------------------------------------")
#Step5: Add the witness version byte (current version is 0):
witver = 0
step5 = [witver] + step4
print("5 Prepend Witness version to result of step 4: ", step5)
print("------------------------------------------------------")
#Step6: Compute the checksum by using the data from step 5 and the H.R.P (bc for MainNet and tb for TestNet)
checksum = bech32.bech32_create_checksum('bc', step5)
print("6 Checksum of result of step 5: ", checksum)
print("------------------------------------------------------")
#Step7: Append the checksum to result of step 5:
step7 = step5+checksum
print("7 Append Checksum to result of step 5: ", step7)
print("------------------------------------------------------")
#Step8: Map each value to its corresponding character in bech32 (qpzry9x8gf2tvdw0s3jn54khce6mua7l)
step8 = [bech32.CHARSET[d] for d in step7]
print("8 map each value to its corresponding character:", step8)
print("------------------------------------------------------")
#Step9: A Bech32_encoded address consists of 3 parts: HRP + Separator + Data:
step9 = "bc" + "1"+ "".join(step8)
print("9 Bitcoin Address: ", step9)
print("------------------------------------------------------")

0 ECDSA Private Key:  1
------------------------------------------------------
1 ECDSA Public Key:  0479be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
 compressed public key 0279be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
------------------------------------------------------
2 SHA256(ECDSA Public Key):  0f715baf5d4c2ed329785cef29e562f73488c8a2bb9dbc5700b361d54b9b0554
------------------------------------------------------
3 RIDEMP160(SHA256(ECDSA Public Key)):  751e76e8199196d454941c45d1b3a323f1433bd6
 RIDEMP160 as array of 8-bit unsigned integers: [117, 30, 118, 232, 25, 145, 150, 212, 84, 148, 28, 69, 209, 179, 163, 35, 241, 67, 59, 214]
------------------------------------------------------
4 RIDEMP160 as array of 5-bit unsigned integers: [14, 20, 15, 7, 13, 26, 0, 25, 18, 6, 11, 13, 8, 21, 4, 20, 3, 17, 2, 29, 3, 12, 29, 3, 4, 15, 24, 20, 6, 14, 30, 22]
-------------------------------